------------------------------------------------------------------------------------------------------------------------------------------------------

# CYTHON TOOLS (for begginers)

This is a python library to process and analyse raw data from the IR02 lab.

Access the [DOCUMENTATION](https://github.com/CIEMAT-Neutrino/CYTHON_TOOLS/wiki) for more information.

You have cloned the repository into a local directory and you are ready to start. 

## PLEASE! When you are done with the tutorial, move to ```macros``` and ```notebooks``` folders to start your own analysis 

In this way, you will be able to keep the tutorial as a reference for future use (or others to use it).

------------------------------------------------------------------------------------------------------------------------------------------------------

## STEP 0: prepare the environment
    
Open a terminal and run to prepare the packages needed for the macros:
```bash
cd scripts
sh setup.sh 
```
Alternatevely, you can open /scripts/requirements.txt and install them manually.

Download the data:
```bash
sh copy_data.sh AFS_USER pcae
```

You will find template data in the ```data/TUTORIAL/raw``` folder which will be used to run the tutorial

## STEP 1: process the raw data


```bash
cd 01TemplateMacros
python3 00Raw2Np.py      TUTORIAL
```
<img src="images/00Raw2Np.png" alt="Raw2Np"/>


```bash
python3 01PreProcess.py     TUTORIAL
```
<img src="images/01PreProcess.png" alt="PreProcess"/>


```bash
python3 02Process.py     TUTORIAL
```
<img src="images/02Process.png" alt="Process"/>

## STEP 2: visualize data

In [1]:
import sys; sys.path.insert(0, '../'); from lib import *;

You have latex installed!. Applying default configuration (tex=True)


In [2]:
# Set options for general visualitation
OPT  = {
    "MICRO_SEC":   True,
    "NORM":        False,                # Runs can be displayed normalised (True/False)
    "ALIGN":       True,
    "LOGY":        False,               # Runs can be displayed in logy (True/False)
    "SHOW_AVE":    "",             # If computed, vis will show average (AveWvf,AveWvfSPE,etc.)
    "SHOW_PARAM":  False,                 # Print terminal information (True/False)
    "CHARGE_KEY":  "ChargeAveRange",     # Select charge info to be displayed. Default: "ChargeAveRange" (if computed)
    "PEAK_FINDER": False,                # Finds possible peaks in the window (True/False)
    "LEGEND":      True,                # Shows plot legend (True/False)
    "SHOW":        True
    }

In [3]:
info     = read_input_file("TUTORIAL")
calib_runs     = [1]
laser_runs     = [9]
noise_runs     = [17]
alpha_runs     = [25]
muons_runs     = [29]

channels = [0,6]



dict_keys(['TYPE', 'MODEL', 'BITS', 'DYNAMIC_RANGE', 'SAMPLING', 'RAW_DATA', 'PATH', 'MONTH', 'OV_LABEL', 'CALIB_RUNS', 'LIGHT_RUNS', 'NOISE_RUNS', 'ALPHA_RUNS', 'MUONS_RUNS', 'CHAN_LABEL', 'CHAN_TOTAL', 'CHAN_POLAR', 'CHAN_AMPLI', 'LOAD_PRESET', 'SAVE_PRESET', 'REF', 'I_RANGE', 'F_RANGE'])




In [4]:
my_runs_calib = load_npy(calib_runs,channels,preset="EVA",info=info,compressed=True)
my_runs_laser = load_npy(laser_runs,channels,preset="EVA",info=info,compressed=True)
my_runs_alpha = load_npy(alpha_runs,channels,preset="EVA",info=info,compressed=True)
my_runs_muons = load_npy(muons_runs,channels,preset="EVA",info=info,compressed=True)
my_runs_noise = load_npy(noise_runs,channels,preset="EVA",info=info,compressed=True)

time = np.arange(len(my_runs_calib[calib_runs[0]][channels[0]]["AveWvf"][0]))*4e-9

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!



## INDIVIDUAL EVENTS

By running the next function you can dynamically visualize the RAW events of the selected run to see everything is OK.

(In future versions we will make possible to visualize any kind of event with the same function.)



```bash
%run run ../01TemplateMacros/0YVisEvent.py
```

In [6]:
!pwd

/home/laura/Analysis_Laura/CYTHON_TOOLS/00Tutorial


In [7]:
%run ../01TemplateMacros/0YVisEvent.py

## NOISE 

In [ ]:
noise = my_runs_noise[noise_runs[0]][channels[0]]["AveWvf"][0]
sp = np.fft.rfft(noise)
freq = np.fft.rfftfreq(5000, d = 4e-9)

fig = px.line(x=freq, y=sp.imag)
custom_plotly_layout(fig, title='FFT', xaxis_title='Frequency [Hz]', yaxis_title='FFT').show()


## LASER

In [ ]:
laser = my_runs_laser[laser_runs[0]][channels[0]]["AveWvf"][0]
laser = laser / np.max(laser)

ref_max_idx = np.argmax(laser)
idx_move = np.argmax(laser)
laser = shift(laser, ref_max_idx-idx_move, cval = 0)

df_laser = pd.DataFrame({"Time":time,"Laser":laser})

In [ ]:
fig_laser = px.line(df_laser, x="Time", y=["Laser"], height=700, width = 900, line_shape = "hv", log_y = False, template="presentation")
fig_laser.update_traces(line_width=2) # Modifica el grosor de las líneas
custom_legend_name(fig_laser,["Laser_1"])
custom_plotly_layout(fig_laser,"Time","Normalized ADC","Laser signals").show()

## ALPHA

In [ ]:
alpha = my_runs_alpha[alpha_runs[0]][channels[0]]["AveWvf"][0]
alpha = alpha / np.max(alpha)

ref_max_idx = np.argmax(alpha)
idx_move = np.argmax(alpha)
alpha = shift(alpha, ref_max_idx-idx_move, cval = 0)

df_alpha = pd.DataFrame({"Time":time,"Alpha":alpha})

In [ ]:
fig_alpha = px.line(df_alpha, x="Time", y=["Alpha"], height=700, width = 900, line_shape = "hv", log_y = False, template="presentation")
fig_alpha.update_traces(line_width=2) # Modifica el grosor de las líneas
custom_legend_name(fig_alpha,["Alpha_1"])
custom_plotly_layout(fig_alpha,"Time","Normalized ADC","Alpha signals").show()

## PedSTD

In [ ]:
AlphaNoise = my_runs_alpha[alpha_runs[0]][channels[0]]["AveWvfPedestal"][0]
df_alphanoise = pd.DataFrame({"Time":time,"AlphaNoise":AlphaNoise})

KeyError: 'AveWvfPedestal'

In [ ]:
fig_alphanoise = px.line(df_alphanoise, x="Time", y=["AlphaNoise"], height=700, width = 900, line_shape = "hv", log_y = False, template="presentation")
fig_alphanoise.update_traces(line_width=2) # Modifica el grosor de las líneas
custom_legend_name(fig_alphanoise,["AlphaNoise_1"])
custom_plotly_layout(fig_alphanoise,"","Normalized ADC","Alpha Noise").show()

NameError: name 'df_alphanoise' is not defined